Now we'll create a chatbot that is able to take information from files and answer questions about them. As we did with the basic math chat bot we first install openai, import it, define the api key, define the client object and finally define the assistant. This time we're using the "file_search" tool and I have used the 2nd year F2A quantum notes for this.

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI

In [ ]:
# Define your OpenAI API key
openai_api_key = ""

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=openai_api_key)
 
assistant = client.beta.assistants.create(
  name="Quantum teacher",
  instructions="You are an expert Quantum physicist. Use you knowledge base to answer questions about quantum physics.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

Next we need to create a vector store. Vector stores are essentially massive vectors which files can be turned into which the AI is then able to interpret. 

(Side tangent that is not totally nessecary for understanding this code but is useful to know: the way chatbots work is through the comparison of these vectors. Lets say for example you took the word "amazon" and on a scale of 0 to 1 you compared it to the word "beach" where 0 is as close as possible and 1 is as far away as possible you might get a value of something like 0.3. You may then compare it to the word "jungle" and get 0.1 but you could also compare it to the word "shopping" and get 0.07. The point is the space that contains all these vectors makes important of the semantics of words and its through this comparison that AI is able to generate answers. Although not explicitly stated in the following code what the AI is doing is it is breaking up the source document into several hundred "chunks" all of equal character length. It then compares the vectors of the chunks against the vectors of our instructions and promts and picks the most similar chunks. Then it feeds these chunks into the chatbot which it then bases its answers on.) 

In [ ]:
vector_store = client.beta.vector_stores.create(name="Quantum")

We then prep all the files that we want to upload to OpenAI by stating the file paths, in this case because the "quantumnotes.txt" is a sibling to the jupyter notebook file we can simple state the file name. Make note of the fact that file_paths is a list and so several files could be included usually however these should be .txt, .pdf or .md. file_streams is a python list of file objects opened in binary read mode, essentially it just makes it so we can turn it into a vector store.

In [ ]:
file_paths = ["quantumnotes.txt"]
file_streams = [open(path, "rb") for path in file_paths]

We then use the an extenstion of the client to upload the file_streams, we make sure to define the vector store that we want to upload the files to and the files that we want to upload. We then also print the status and counts of the files as they upload so that we can ensure that everything has gone through properly.

In [ ]:
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
print(file_batch.status)
print(file_batch.file_counts)

As we did in the simple maths chatbot we define an assistant with the relavent id and tool resources this time making sure to give it access to the vector store we just created.

In [ ]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Here we can also upload the file seperately to OpenAI so that its id can be used in a message.

In [ ]:
message_file = client.files.create(
  file=open("quantumnotes.txt", "rb"), purpose="assistants"
)

Again as we did previously we create a thread this time for speed we can include the message inside the thread, the only thing being different is the "attachments" which takes the id of the message_file so it knows which vector store to use and the tools so it knows to use file_search. Again we can print a log of this happening.

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "what is degenerate perturbation theory",
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
print(thread.tool_resources.file_search)

Again as we did previously we define an event handler this time there is only one new function which is "on_message_done" simply provides a citation to all of the files used. In our case there is only one file so several citations may lead back to the same file but in the case of uploading several files to the vector store, this would provide citations to each of the files.

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

Once again we do as we did before inputing the relavent ids and such. Also excuse the horribly formatted mathematics it makes, I will try to improve this.

In [ ]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Alex.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()